# **Celebrity Search Using Llama-2 7B Model**

Necessary Libraries required for Setup:

1. langchain: Required as a framework to interact with Large Language Model
2. Ctransformers: Used in loading the model effectively on CPU Compute
3. Bitsandbytes: loads the model in bits( 8 bytes)

In [1]:
!pip install -q langchain ctransformers  bitsandbytes

In [2]:
from langchain.llms import CTransformers
from langchain.chains import LLMChain, SequentialChain
from langchain.memory import ConversationBufferMemory
from langchain import HuggingFacePipeline
from langchain import PromptTemplate
from langchain.agents import AgentType,initialize_agent,load_tools

In [3]:
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
llm = CTransformers(model = 'TheBloke/Llama-2-7B-Chat-GGML',model_file = 'llama-2-7b-chat.ggmlv3.q2_K.bin',callback = [StreamingStdOutCallbackHandler])

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

In [4]:
# dob_memory.load_memory_variables({'name':''})

NameError: ignored

In [5]:
#  Creation of memory units where we store outputs of each of the chain and use them as inputs in prompts for the next chain in sequence
person_memory = ConversationBufferMemory(input_key='name', memory_key='person_history')
dob_memory = ConversationBufferMemory(input_key='person', memory_key='dob_history')
descr_memory = ConversationBufferMemory(input_key='dob', memory_key='description_history')

# Prompt template for the LLM Model based on which it is asked questions
first_input_prompt=PromptTemplate(
    input_variables=['name'],
    template="Who is {name}?"
)

# LLM Chain is used to execute the prompt template
# chain=LLMChain(
#     llm=llm,prompt=first_input_prompt,verbose=True)

# Actual Code
chain=LLMChain(
    llm=llm,prompt=first_input_prompt,verbose=True,output_key='person',memory=person_memory)

# chain.run('Sachin Tendulkar')

second_input_prompt=PromptTemplate(
    input_variables=['person'],
    template="when was {person} born"
)

chain2=LLMChain(
    llm=llm,prompt=second_input_prompt,verbose=True,output_key='dob',memory=dob_memory)

third_input_prompt=PromptTemplate(
    input_variables=['dob'],
    template="Mention 5 major events happened around {dob} in the world"
)
chain3=LLMChain(llm=llm,prompt=third_input_prompt,verbose=True,output_key='description',memory=descr_memory)

parent_chain=SequentialChain(
    chains=[chain,chain2,chain3],input_variables=['name'],output_variables=['person','dob','description'],verbose=True)


parent_chain({'name':'Sachin Tendulkar'})



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Who is Sachin Tendulkar?

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
when was 

Sachin Ramesh Tendulkar (born April 24, 1970) is a former Indian cricketer. He was named one of the greatest batsmen in cricket history and earned the moniker "Master Blaster." His exceptional talent on the field brought him global recognition and adoration from fans worldwide.

Tendulkar played domestic cricket for his home state, Mumbai, before joining the national side. At age 14, he made his first-class debut in 1989 and quickly rose through the ranks as an Indian player. He established himself as a top-order batsman and scored over 34,000 runs throughout his career, making him the highest run-scorer in domestic cricket history.

Tendulkar received numerous accolades for his outstanding play on the field. He was named India's Player of the Year on several occasions and

{'name': 'Sachin Tendulkar',
 'person': '\n\nSachin Ramesh Tendulkar (born April 24, 1970) is a former Indian cricketer. He was named one of the greatest batsmen in cricket history and earned the moniker "Master Blaster." His exceptional talent on the field brought him global recognition and adoration from fans worldwide.\n\nTendulkar played domestic cricket for his home state, Mumbai, before joining the national side. At age 14, he made his first-class debut in 1989 and quickly rose through the ranks as an Indian player. He established himself as a top-order batsman and scored over 34,000 runs throughout his career, making him the highest run-scorer in domestic cricket history.\n\nTendulkar received numerous accolades for his outstanding play on the field. He was named India\'s Player of the Year on several occasions and was given the Arjuna Award by the government of India in 1997. Throughout his career, he won many Man of the Match awards and took home the ICC Cricket World Cup\'s B